In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# kaggle competitions download -c nyc-taxi-trip-duration

In [ ]:
import pandas as pd
from scipy.stats import beta
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.metrics import mean_squared_error

In [ ]:
data_train=pd.read_csv("/kaggle/input/big-data-duration/train.csv")
data_train

In [ ]:
data_train.info()

In [ ]:
data_train.isnull().sum()

In [ ]:
data_train.describe()

In [ ]:
# data['trip_duration_min'] = data['trip_duration'] / 60
# data['trip_duration_houre'] = data['trip_duration_min'] / 60
# data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# حساب معامل الارتباط مع target
correlation = data_train.corr(numeric_only=True)['trip_duration'].sort_values(ascending=False)

# رسم مخطط bar
plt.figure(figsize=(10, 6))
sns.barplot(x=correlation.values, y=correlation.index, palette='coolwarm')
plt.title('Correlation with Target')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.show()


In [ ]:
#cal querter 1 &3
Q1 = data_train['trip_duration'].quantile(0.25)
Q3 = data_train['trip_duration'].quantile(0.75)
IQR = Q3 - Q1

# specific lower & upper
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# extract abnormal value
outliers = data_train[(data_train['trip_duration'] < lower_bound) | (data_train['trip_duration'] > upper_bound)]

# view rseult
print(" number abnormal value:", outliers.shape[0])
print("abnormal value:")
print(outliers[['trip_duration']].head(50))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.boxplot(data=data_train, y='trip_duration')
plt.axhline(lower_bound, color='red', linestyle='--', label='Lower Bound')
plt.axhline(upper_bound, color='red', linestyle='--', label='Upper Bound')
plt.title('Boxplot of trip_duration with Outlier Bounds')
plt.legend()
plt.show()


In [ ]:
skew_value = data_train['trip_duration'].skew()
print(f"Skewness: {skew_value}")


In [ ]:
import numpy as np


data_train['log_trip_duration'] = np.log1p(data_train['trip_duration'])


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data_train['log_trip_duration'], kde=True)
plt.title("Distribution after log transformation")
plt.show()

# Skewness بعد التحويل
print("New skewness:", data_train['log_trip_duration'].skew())


In [ ]:
data_train.columns

In [ ]:
def new_feature(data_train):
    pickup_datetime=pd.to_datetime(data_train["pickup_datetime"])
    data_train['pickup_month']= pickup_datetime.dt.month
    # data_train['pickup_year']= pickup_datetime.dt.year
    # data_train['pickup_day']= pickup_datetime.dt.day
    data_train['pickup_hour']= pickup_datetime.dt.hour
    # data_train['pickup_minute']= pickup_datetime.dt.minute
    data_train['pickup_dayofyear']= pickup_datetime.dt.dayofyear
    data_train['pickup_dayofweek']= pickup_datetime.dt.dayofweek
    # data_train['pickup_weekend'] = (pickup_datetime.dt.weekday >= 5).astype(int)
    # data_train['is_rush_hour']=((data_train['pickup_hour']>=7)&(data_train['pickup_hour']<=9)|(data_train['pickup_hour'] >=17)&(data_train['pickup_hour'] <=19)).astype(int)
    # data_train['is_night']=((data_train['pickup_hour']>=22)|(data_train['pickup_hour']<=5)).astype(int)
    data_train['euclidean_distance']=np.sqrt(np.square(data_train['pickup_longitude'] -data_train['dropoff_longitude'])+np.square(data_train['pickup_latitude'] - data_train['dropoff_latitude']))
    # data_train['Manhattan_distance']=np.abs(data_train['pickup_longitude'] -data_train['dropoff_longitude'])+np.abs(data_train['pickup_latitude'] - data_train['dropoff_latitude'])
    # data_train['center_latitude']=(data_train['pickup_latitude'] +data_train['dropoff_latitude'])/2
    # data_train['center_longitude']=(data_train['pickup_longitude'] +data_train['dropoff_longitude'])/2
    # data_train['direction']=np.degrees(np.arctan2((data_train['dropoff_latitude'] -data_train['pickup_latitude']),(data_train['dropoff_longitude'] -data_train['pickup_longitude'])))
    data_train['store_and_fwd_flag']=np.where(data_train['store_and_fwd_flag']=='N',0,1)
    return data_train

In [ ]:
data_newfeature=new_feature(data_train)
 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# حساب معامل الارتباط مع target
correlation = data_newfeature.corr(numeric_only=True)['log_trip_duration'].sort_values(ascending=False)

# رسم مخطط bar
plt.figure(figsize=(10, 6))
sns.barplot(x=correlation.values, y=correlation.index, palette='coolwarm')
plt.title('Correlation with Target')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.show()



In [ ]:
data_newfeature.columns


In [ ]:
from sklearn.model_selection import train_test_split

X = data_newfeature.drop(['id', 'pickup_datetime', 'trip_duration','log_trip_duration'], axis=1) 
y = data_newfeature['log_trip_duration']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_val_scaled = scaler.transform(X_val)

ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_scaled,y_train)
print(f"\nRidge model coefficients: {ridge_model.coef_}")
print(f"Ridge model intercept: {ridge_model.intercept_}")

y_pred_log = ridge_model.predict(X_val_scaled)
y_pred_original_scale = np.expm1(y_pred_log)
y_val_original_scale = np.expm1(y_val) 

rmse = np.sqrt(mean_squared_error(y_val_original_scale, y_pred_original_scale))
r2 = r2_score(y_val_original_scale, y_pred_original_scale)
print(f"R-squared (R2 Score): {r2:.4f}")

r2_log = r2_score(y_val, y_pred_log)
print(f"R-squared on Log Scale: {r2_log:.4f}")



In [ ]:
# joblib.dump(ridge_model, 'ridge_model.pkl')
# joblib.dump(scaler, 'scaler.pkl')


In [ ]:
# ridge_model = joblib.load('ridge_model.pkl')
# scaler = joblib.load('scaler.pkl')
